In [1]:
import pandas as pd
import numpy as np
from pandas import Series,DataFrame

In [2]:
#pandas.merge根据一个或者多个键将不同的DataFrame行连接起来
#类似于数据库中的连接操作

In [3]:
df1 = DataFrame({'key':['b','b','a','c','a','a','b'],
                 'data1':range(7)})
df2 = DataFrame({'key':['a','b','d'],
                 'data2':range(3)})
print(df1)
print(df2)

  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   a      5
6   b      6
  key  data2
0   a      0
1   b      1
2   d      2


In [5]:
print(pd.merge(df1,df2))
#默认是使用公共列上的数据的交集作为基准 进行合并
pd.merge(df1,df2,on='key')
#使用on参数可以显示定义按照指定列的交集进行合并

  key  data1  data2
0   b      0      1
1   b      1      1
2   b      6      1
3   a      2      0
4   a      4      0
5   a      5      0


,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [7]:
df3 = DataFrame({'lkey':['b','b','a','c','a','a','b'],
                 'data1':range(7)})
df4 = DataFrame({'rkey':['a','b','d'],
                 'data2':range(3)})
print(df3)
print(df4)

  lkey  data1
0    b      0
1    b      1
2    a      2
3    c      3
4    a      4
5    a      5
6    b      6
  rkey  data2
0    a      0
1    b      1
2    d      2


In [9]:
#pd.merge(df3,df4) 如果没有公共列 则merge会报错
pd.merge(df3,df4,left_on='lkey',right_on='rkey')
#设置left_on和right_on的意思是取left_on和right_on的交集 进行合并

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [11]:
#默认情况下，merge使用的是内连接，即指定列求交集的连接方式
#使用how参数可以显式确定合并方法
pd.merge(df1,df2,how='outer')
#ounter是按照列的交集进行全连接 一方没有的用NaN代替

  key  data1  data2
0   b    0.0    1.0
1   b    1.0    1.0
2   b    6.0    1.0
3   a    2.0    0.0
4   a    4.0    0.0
5   a    5.0    0.0
6   c    3.0    NaN
7   d    NaN    2.0


,key,data1,data2
0,b,0,1.0
1,b,1,1.0
2,a,2,0.0
3,c,3,NaN
4,a,4,0.0
5,a,5,0.0
6,b,6,1.0


In [14]:
df5 = DataFrame({'key':['b','b','a','c','a','a','b'],
                 'data1':range(7)})
df6 = DataFrame({'key':['a','b','d','b','a'],
                 'data2':range(5)})
print(df5)
print(df6)
pd.merge(df5,df6,on='key')

  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   a      5
6   b      6
  key  data2
0   a      0
1   b      1
2   d      2
3   b      3
4   a      4


,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,6,1
5,b,6,3
6,a,2,0
7,a,2,4
8,a,4,0
9,a,4,4


In [13]:
pd.merge(df5,df6,how='left',on='key')
#多对多连接产生的结果是行的笛卡尔积，df5有三行b，df6有两行b,最终的结果就是6行b 形成笛卡尔积
#而使用left可以保留第一个有第二个没有的行，设置成NaN 而同理right则之保留第一个没有第二个有的行

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,4.0
6,c,3,NaN
7,a,4,0.0
8,a,4,4.0
9,a,5,0.0


In [17]:
left = DataFrame({'key1':['foo','foo','bar'],
                  'key2':['one','two','three'],
                 'lval':[1,2,3]})
right= DataFrame({'key1':['foo','foo','bar','bar'],
                  'key2':['one','one','one','two'],
                 'lval':[4,5,6,7]})
print(left)
print(right)

  key1   key2  lval
0  foo    one     1
1  foo    two     2
2  bar  three     3
  key1 key2  lval
0  foo  one     4
1  foo  one     5
2  bar  one     6
3  bar  two     7


In [18]:
#按照多个相同键进行连接
pd.merge(left,right,on=['key1','key2'],how='outer')

#  在进行列-列合并的时候，如果不是选择的outer全连接，那么可能会导致行索引的丢弃

,key1,key2,lval_x,lval_y
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,three,3.0,NaN
4,bar,one,NaN,6.0
5,bar,two,NaN,7.0


In [20]:
#如果面临有多个重复键但是仅想要通过部分重复键合并的情况
print(pd.merge(left,right,on='key1'))
#此时由于left和right都有重名但无需合并的key2列，默认会加_x _y 也可以使用参数修改
pd.merge(left,right,on='key1',suffixes=('_left','_right'))

  key1 key2_x  lval_x key2_y  lval_y
0  foo    one       1    one       4
1  foo    one       1    one       5
2  foo    two       2    one       4
3  foo    two       2    one       5
4  bar  three       3    one       6
5  bar  three       3    two       7


,key1,key2_left,lval_left,key2_right,lval_right
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,three,3,one,6
5,bar,three,3,two,7


In [21]:
print('-----------------------------')
#索引上的合并

-----------------------------


In [22]:
left1 = DataFrame({'key':['a','b','a','a','b','c'],
                   'value':range(6)})
right1 = DataFrame({'group_val':[3.5,7]},index=['a','b'])
print(left1)
print(right1)

  key  value
0   a      0
1   b      1
2   a      2
3   a      3
4   b      4
5   c      5
   group_val
a        3.5
b        7.0


In [24]:
#lef1和right1有公共部分 但是left1是key列的a b 而right1是行 也就是索引上的a b
#如果要将一个行一个列进行合并 使用left_index/right_index参数
pd.merge(left1,right1,left_on='key',right_index=True,how='left')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [26]:
#层次索引的合并
lefth = DataFrame({'key1':['Ohio','Ohio','Ohio','Nevada','Nevada'],
                   'key2':[2000,2001,2002,2001,2002],
                 'data':np.arange(5.)})
righth = DataFrame(np.arange(12).reshape((6,2)),
                   index=[['Nevada','Nevada','Ohio','Ohio','Ohio','Ohio'],
                          [2001,2000,2000,2000,2001,2002]],
                   columns=['event1','event2'])
print(lefth)
print(righth)

     key1  key2  data
0    Ohio  2000   0.0
1    Ohio  2001   1.0
2    Ohio  2002   2.0
3  Nevada  2001   3.0
4  Nevada  2002   4.0
             event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11


In [28]:
#此时的合并是将lefth的key1 key2上的[州-年]对和righth的双层index对进行合并
pd.merge(lefth,righth,how='outer',left_on=['key1','key2'],right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [30]:
df7 = DataFrame({'lkey':['b','b','a','c',],
                 'data':range(4)},index=['one','two','three','four'])
df8 = DataFrame({'rkey':['a','b','d'],
                 'data':range(3)},index=['four','five','one'])
print(df7)
print(df8)

      lkey  data
one      b     0
two      b     1
three    a     2
four     c     3
     rkey  data
four    a     0
five    b     1
one     d     2


In [31]:
pd.merge(df7,df8,how='outer',left_index=True,right_index=True)
#两边都使用index进行合并时 合并出的index就是原来的index的交集（并集）
#而列上如果存在同名的列，由于无需合并 所以重命名了_x _y

,lkey,data_x,rkey,data_y
five,NaN,NaN,b,1.0
four,c,3.0,a,0.0
one,b,0.0,d,2.0
three,a,2.0,NaN,NaN
two,b,1.0,NaN,NaN


In [34]:
#可以使用join方法进行索引合并
df7.join(df8,lsuffix='_left',rsuffix='_right')
#join默认实现的是左连接即调用者的index会完全保留

,lkey,data_left,rkey,data_right
one,b,0,d,2.0
two,b,1,NaN,NaN
three,a,2,NaN,NaN
four,c,3,a,0.0


In [35]:
#不过同样可以使用how参数进行修改连接方式
df7.join(df8,lsuffix='_left',rsuffix='_right',how='outer')
#得到的效果和 pd.merge(df7,df8,how='outer',left_index=True,right_index=True)一样

,lkey,data_left,rkey,data_right
five,NaN,NaN,b,1.0
four,c,3.0,a,0.0
one,b,0.0,d,2.0
three,a,2.0,NaN,NaN
two,b,1.0,NaN,NaN


In [36]:
#同样地也可以实现调用函数的index和被调用函数的column的合并
left1.join(right1,on='key')
#效果同 pd.merge(left1,right1,left_on='key',right_index=True,how='left')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [38]:
print("###################################################")
#上述的连接是面向数据库风格 合并形式笛卡尔积形式的连接

#轴向连接 更像是一种拼接 无交集的纯连接

###################################################


In [39]:
s1 = Series([0,1],index=['a','b'])
s2 = Series([2,3,4],index=['c','d','e'])
s3 = Series([5,6],index=['f','g'])
print(s1)
print(s2)
print(s3)

a    0
b    1
dtype: int64
c    2
d    3
e    4
dtype: int64
f    5
g    6
dtype: int64


In [40]:
pd.concat([s1,s2,s3])
#将三个Series进行拼接得到的结果

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [41]:
#还可以按照别的轴进行拼接 Series按照列进行拼接得到的是DataFrame
pd.concat([s1,s2,s3],axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [43]:
s4 = pd.concat([s1*5,s3])
s4

a    0
b    5
f    5
g    6
dtype: int64

In [44]:
pd.concat([s1,s4],axis=1)

,0,1
a,0.0,0
b,1.0,5
f,NaN,5
g,NaN,6


In [45]:
pd.concat([s1,s4],axis=1,join='inner')
#concat的join参数默认为outer是一种取并集的连接即为拼接
#而设置join为inner 则为取交集的拼接

,0,1
a,0,0
b,1,5


In [49]:
result = pd.concat([s1,s2,s3],keys=['one','two','three'])
result
#对每个Series创建了更高一层的索引 使用keys参数

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

In [50]:
result.unstack()
#通过unstack可以将层次化索引分开成DataFrame

,a,b,c,d,e,f,g
one,0.0,1.0,NaN,NaN,NaN,NaN,NaN
two,NaN,NaN,2.0,3.0,4.0,NaN,NaN
three,NaN,NaN,NaN,NaN,NaN,5.0,6.0


In [51]:
df_1 = DataFrame(np.arange(6).reshape(3,2),index=['a','b','c'],
                 columns=['one','two'])
df_2 = DataFrame(5+np.arange(4).reshape(2,2),index=['a','c'],
                 columns=['three','four'])
print(df_1)
print(df_2)

   one  two
a    0    1
b    2    3
c    4    5
   three  four
a      5     6
c      7     8


In [53]:
pd.concat([df_1,df_2]) #DataFrame同理 是一种拼接操作

,one,two,three,four
a,0.0,1.0,NaN,NaN
b,2.0,3.0,NaN,NaN
c,4.0,5.0,NaN,NaN
a,NaN,NaN,5.0,6.0
c,NaN,NaN,7.0,8.0


In [54]:
pd.concat([df_1,df_2],axis=1)

,one,two,three,four
a,0,1,5.0,6.0
b,2,3,NaN,NaN
c,4,5,7.0,8.0


In [58]:
result2 = pd.concat([df_1,df_2],axis=1,keys=['level1','level2'])
result2

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [61]:
pd.concat([df_1,df_2],axis=1,keys=['level1','level2'],
          names=['upper','lower'])
#names可以设置表头

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [62]:
#忽略行索引的合并
df_3 = DataFrame(np.random.randn(3,4),columns=['a','b','c','d'])
df_4 = DataFrame(np.random.randn(2,3),columns=['b','d','a'])
print(df_3)
print(df_4)

          a         b         c         d
0  0.021792  0.245787  1.154943  1.515377
1  0.373992 -0.765767 -0.998321 -0.379294
2  0.262254 -0.553084 -0.982880  0.205100
          b         d         a
0 -0.327762 -0.910637 -0.909360
1 -0.309139  0.119290 -2.013493


In [64]:
pd.concat([df_3,df_4])

,a,b,c,d
0,0.021792,0.245787,1.154943,1.515377
1,0.373992,-0.765767,-0.998321,-0.379294
2,0.262254,-0.553084,-0.982880,0.205100
0,-0.909360,-0.327762,NaN,-0.910637
1,-2.013493,-0.309139,NaN,0.119290


In [65]:
#如果要忽略index使用参数ignore_index来进行即可
pd.concat([df_3,df_4],ignore_index=True)

,a,b,c,d
0,0.021792,0.245787,1.154943,1.515377
1,0.373992,-0.765767,-0.998321,-0.379294
2,0.262254,-0.553084,-0.982880,0.205100
3,-0.909360,-0.327762,NaN,-0.910637
4,-2.013493,-0.309139,NaN,0.119290


In [66]:
print("############################################")
#除了连接和拼接以外
#第三种情况是 取a的一部分和b的一部分进行合并 往往带有判断条件

############################################


In [67]:
a = Series([np.nan,2.5,np.nan,3.5,4.5,np.nan],index=['f','e','d','c','b','a'])
b = Series(np.arange(len(a)),dtype=np.float64,index=['f','e','d','c','b','a'])
print(a)
print(b)

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64
f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    5.0
dtype: float64


In [72]:
pd.concat([a,b],axis=1,join='inner')

,0,1
f,NaN,0.0
e,2.5,1.0
d,NaN,2.0
c,3.5,3.0
b,4.5,4.0
a,NaN,5.0


In [73]:
#如果要在某时刻取a中的值 某时刻取b中的值
#使用np.where
np.where(pd.isnull(a),b,a)

array([0. , 2.5, 2. , 3.5, 4.5, 5. ])